In [ ]:
!pip install scikit-plot
!pip install jax-unirep

Try to deal with GPU memory 
accroading to https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html

In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
from jax_unirep import get_reps
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params
import os
import gc

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='False'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.70'

/home/kongkitimanonk/.conda/envs/jupyter_NB/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
os.environ['MKL_NUM_THREADS']='1' 
os.environ['OPENBLAS_NUM_THREADS']='1'

os.environ["NUM_INTER_THREADS"]="1"
os.environ["NUM_INTRA_THREADS"]="1"
os.environ.update(
        XLA_FLAGS=(
            '--xla_cpu_multi_thread_eigen=false '
            'intra_op_parallelism_threads=2 '
            'inter_op_parallelism_threads=2 '
            'OMP_NUM_THREADS=1  '
        ),
        XLA_PYTHON_CLIENT_PREALLOCATE='false',
    )
#os.environ["XLA_FLAGS"] = ("--xla_cpu_multi_thread_eigen=false intra_op_parallelism_threads=10")

In [3]:
fastas = "/home/kongkitimanonk/SCRATCH_NOBAK/cd-hit/bk_fasta/SRR11234331.assembly.len15.fasta"
plk = "/home/kongkitimanonk/SCRATCH_NOBAK/phase3/PoisonFrog.len15.pkl"

In [5]:
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))

In [6]:
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,0,VAQHRPLIPVLPERPQYSGRSLHSPAAVSMPLSDLDLLAVTDLSLS...,50
1,1,MERIELRHANSEEGRSQKTSCRYKFF,28
2,2,MQTLKRGGHRRLAVDTNFSEKPED,26
3,3,MPQLNSLHTSVSLSVTPL,20
4,4,MYCLAMLHPTQVRGR,17
...,...,...,...
12753725,12753725,MFFSSPSRRVFSLSNESS,20
12753726,12753726,MNPITPICTVHSTSIAFLFFILAV,26
12753727,12753727,CGLWGGQSVRTQRQLVMSDAQHVPSPAAGSELVSVLWAVGRQLLLL...,163
12753728,12753728,MPRAMRKRHRSRKRES,18


Remove Duplicate

In [7]:
df.drop_duplicates(subset=['Sequence'],inplace=True)
df

,ID,Sequence,length
0,0,VAQHRPLIPVLPERPQYSGRSLHSPAAVSMPLSDLDLLAVTDLSLS...,50
1,1,MERIELRHANSEEGRSQKTSCRYKFF,28
2,2,MQTLKRGGHRRLAVDTNFSEKPED,26
3,3,MPQLNSLHTSVSLSVTPL,20
4,4,MYCLAMLHPTQVRGR,17
...,...,...,...
12753717,12753717,EPQRRSARLSAKPAPPKAEPKPKKPPAAKKADKAQKRKKGKADSGKDA,48
12753718,12753718,HLFQNLPFLFFFFELCQPSLLLVAFWALALLLEGRV,37
12753720,12753720,PEAKVGGLCSRWKDSVFVRMVLELKVTVRHPRLRIWSLWWIFNSAG...,97
12753721,12753721,MKKVAILQENALLLIN,18


In [21]:
#0-613834
#613834-1277668
#1277668-1841502
#1841502-2455336
#2455336 -3069170
# 3069170-3683004
#3683004-4296838
# 4296838 - 4910674 
#df1 = df.iloc[0:300000, :]
#df2 = df.iloc[300000:613834, :]

df3 = df.iloc[613834:1277668, :]
#df3 = df.iloc[1277668:1841502, :]
#df4 = df.iloc[1841502:2455336, :]
#df5 = df.iloc[2455336:3069170, :]
#df6 = df.iloc[3069170:3683004, :]
#df7 = df.iloc[3683004:4296838, :]
#df8 = df.iloc[4296838:4910674, :]

In [22]:
df3

,ID,Sequence,length
1117804,1117804,MIWVCMSGKGTGEMAVITSSINAQVYVDILDTFLVLSIERMFGDDE...,114
1117809,1117809,HAICAIRASAIICMSCHCVPSCKGFPTLGTDKSCGNEAWIVVSLEM...,120
1117812,1117812,VWGPASPGRLFAVGVDTGVWCVLFDGAAGWGPVRRQFFRLETLVCL...,54
1117814,1117814,MSNHFLSSYTNCTCLATEG,21
1117815,1117815,LGLCCAYRTWPLYCECLTIFLAPTLIVHVWPQKDESCILGSLL,44
...,...,...,...
2582399,2582399,KKKRNRAALGRYARMEKLAIEGIVVQRAECRPAASEHYMQMKRRQI...,164
2582400,2582400,MSNTCCLSARARFPSSFFFFSYSILFAWLDTGL,35
2582402,2582402,LAAELERRGEAAGAEIPESGVRAAAGEHRAGLLLRLFGRFSSWKANH,48
2582411,2582411,PSIAMKGLMICLVAAAFCMDLARSLQCYYCKEESDLAKCNGVKECS...,356


In [10]:
def createREPs(df, filename):
    _h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
    df['reps']=_h_avg.tolist() # if there is a problem , might be here , possible solution is reindexing
    df.to_pickle(filename) # dont forget to change the file name to 0_613834

In [11]:
createREPs(df1,"/home/kongkitimanonk/SCRATCH_NOBAK/phase3/PoisonFrog.len15.0_300000.pkl")

<ipython-input-10-204853ba1859>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reps']=_h_avg.tolist() # if there is a problem , might be here , possible solution is reindexing


In [13]:
df1=""
createREPs(df2,"/home/kongkitimanonk/SCRATCH_NOBAK/phase3/PoisonFrog.len15.300000_613834.pkl")

<ipython-input-10-204853ba1859>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reps']=_h_avg.tolist() # if there is a problem , might be here , possible solution is reindexing


In [ ]:
df2=""
createREPs(df3,"/home/kongkitimanonk/SCRATCH_NOBAK/phase3/PoisonFrog.len15.613834_1277668.pkl")

----- base ---

In [ ]:
_h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
df['reps']=_h_avg.tolist()
df

In [ ]:
df.to_pickle(plk)
